!kg download # after configuring Kaggle CLI, this will download the dataset  
!unzip -q images_training_rev1.zip

In [33]:
!time unzip -q images_training_rev1.zip

8.48user 1.81system 0:10.31elapsed 99%CPU (0avgtext+0avgdata 4000maxresident)k
0inputs+1883320outputs (0major+1000minor)pagefaults 0swaps


In [1]:
import pandas as pd
import zipfile

with zipfile.ZipFile('training_solutions_rev1.zip') as zfile:
    with zfile.open('training_solutions_rev1.csv') as f:
        df = pd.read_csv(f)
df.head()

,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,100008,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,...,0.000000,0.279952,0.138445,0.000000,0.000000,0.092886,0.0,0.0,0.0,0.325512
1,100023,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,...,0.018764,0.000000,0.131378,0.459950,0.000000,0.591328,0.0,0.0,0.0,0.000000
2,100053,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,100078,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,...,0.000000,0.094549,0.000000,0.094549,0.189098,0.000000,0.0,0.0,0.0,0.000000
4,100090,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000


In [3]:
import numpy as np
import os


img_names = df['GalaxyID'].astype('str') + '.jpg'
labels = np.apply_along_axis(np.argmax, 1, df.iloc[:, 1:4].values)


"""
old_path = 'images_training_rev1/'
root_path = 'true_labels/'

# create directories for each of the true image labels
for new_path in set(labels):
    new_path = root_path + str(new_path)
    if not os.path.isdir(new_path):
        print('Making new directory: {}'.format(new_path))
        os.makedirs(new_path)

# move images to correctly labelled directory
for image_name, label in zip(img_names, labels):
    os.rename(old_path + image_name, root_path + str(label) + '/' + image_name)
"""

new_path = 'imgs/' # change for respective data set

for folder_path in ['train', 'val', 'test']:
    for sub_path in set(labels):
        new_dir = new_path + folder_path + '/' + str(sub_path)
        if not os.path.isdir(new_dir):
            print('Making new directory: {}'.format(new_dir))
            os.makedirs(new_dir)


np.random.seed(42)
train_val_test_split = np.random.choice(['train', 'val', 'test'], size=len(df), p=[0.6, 0.2, 0.2])
old_path = 'images_training_rev1/'

for image_name, dataset, label in zip(img_names, train_val_test_split, labels):
    os.rename(old_path + image_name, new_path + dataset + '/' + str(label) + '/' + image_name)

Making new directory: imgs/train/0
Making new directory: imgs/train/1
Making new directory: imgs/train/2
Making new directory: imgs/val/0
Making new directory: imgs/val/1
Making new directory: imgs/val/2
Making new directory: imgs/test/0
Making new directory: imgs/test/1
Making new directory: imgs/test/2


In [10]:
"""
from vgg16 import Vgg16
import os

path = '../courses/deeplearning1/nbs/data/distracted_driver/imgs/' # change for respective data set
model_path = '../courses/deeplearning1/nbs/data/distracted_driver/models/' # change for respective data set
batch_size = 64

if not os.path.isdir(model_path): # make validation folder
    print('Making new directory: {}'.format(model_path))
    os.mkdir(model_path)

vgg = Vgg16() # imagenet weights already loaded
# during training, shuffle should be set to true. Or else, it will likely receive training on the same class consecutively
train_batches = vgg.get_batches(path + 'train/', batch_size=batch_size, shuffle=True)
val_batches = vgg.get_batches(path + 'val/', batch_size=batch_size * 2, shuffle=False)
vgg.finetune(train_batches) # optimizer is Adam
"""

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


Found 17943 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.


In [ ]:
vgg.fit(train_batches, val_batches, nb_epoch=1)

In [ ]:
# what does vgg.finetune do? how to do a mixiterator type of thing to get probabilistic inputs